# Supplementary figure for correlated odor concentrations

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import seaborn as sns
import os, json
pj = os.path.join

In [ ]:
do_save_plots = True
# Resources
root_dir = pj("..", "..", "..")
data_folder = pj(root_dir, "results", "for_plots")
panels_folder = "panels/"
params_folder = pj(root_dir, "results", "common_params")

In [ ]:
# rcParams
with open(pj(params_folder, "olfaction_rcparams.json"), "r") as f:
    new_rcParams = json.load(f)
plt.rcParams.update(new_rcParams)

# color maps
with open(pj(params_folder, "back_colors.json"), "r") as f:
    all_back_colors = json.load(f)
back_color = all_back_colors["back_color"]
back_color_samples = all_back_colors["back_color_samples"]
back_palette = all_back_colors["back_palette"]

with open(pj(params_folder, "orn_colors.json"), "r") as f:
    orn_colors = json.load(f)
    
with open(pj(params_folder, "inhibitory_neuron_two_colors.json"), "r") as f:
    neuron_colors = np.asarray(json.load(f))
with open(pj(params_folder, "inhibitory_neuron_full_colors.json"), "r") as f:
    neuron_colors_full = np.asarray(json.load(f))

with open(pj(params_folder, "model_colors.json"), "r") as f:
    model_colors = json.load(f)
with open(pj(params_folder, "model_nice_names.json"), "r") as f:
    model_nice_names = json.load(f)

models = list(model_colors.keys())
print(models)

In [ ]:
# Extra color here
#noise_clr = "xkcd:sage"
#noise_clr = back_palette[0]
#noise_clr = mpl.colors.to_hex("xkcd:ultramarine")
#noise_clr = mpl.colors.to_hex("xkcd:dark sea green")
noise_clr = sns.color_palette("colorblind", n_colors=4)[3]
print(noise_clr)

In [ ]:
n_neu, n_components = np.load(os.path.join(data_folder, "correlation",
    "ibcm_examples_turbulent_correl.npz"))["cbars_gamma_ser_0.2"].shape[1:3]
print(n_neu, n_components)

In [ ]:
# Extra aesthetic parameters for this figure

# More legend rcParams: make everything smaller by 30 %
plt.rcParams["patch.linewidth"] = 0.75
legend_rc = {"labelspacing":0.5, "handlelength":2.0, "handleheight":0.7, 
             "handletextpad":0.8, "borderaxespad":0.5, "columnspacing":2.0}
for k in legend_rc:
    plt.rcParams["legend."+k] = 0.75 * legend_rc[k]

new_color = "r"
linestyles = ["-", "--", ":", (0, (5, 1, 2, 1)), "-."]
neuron_styles = linestyles + [(0, (1, 2, 1, 2))]

In [ ]:
def l2_norm(vecs, axis=-1):
    r""" Computes l2 norm of vectors stored along the last axis of vecs.
    Args:
        vecs can be either a single vector (1d) or an  arbitrary array of vectors,
            where the last dimension indexes elements of vectors.
        axis (int): which axis to sum along.

    Returns: array of distances of same shape as vecs
        except for the summation axis, removed.
    """
    return np.sqrt(np.sum(vecs**2, axis=axis))

## Panel A: illustrate correlated turbulent odors

In [ ]:
np.load(pj(data_folder, "correlation", "ibcm_examples_turbulent_correl.npz")).keys()

In [ ]:
mixed_concs_ser = np.load(pj(data_folder, "correlation", 
                "ibcm_examples_turbulent_correl.npz"))["mixed_concs"]
tser = np.arange(0.0, 1.0*mixed_concs_ser.shape[0], 1.0)
mixed_concs_ser.shape

In [ ]:
tslice = slice(65250, 66250, 1)
t0 = tser[tslice][0]
fig, ax = plt.subplots()
lwidth = 0.75
tscale = 0.01  # 10 ms per step
correl_rho = 0.7
for i in range(n_components):
    if i < 3:
        clr = back_palette[2-i]
        lw = lwidth
        lbl = "Odor {}".format(i)
    else:
        clr = noise_clr
        lw = lwidth * 1.5
        lbl = "Odor {} (cor-\nrelated with {})".format(i, i-1)
    ax.plot((tser[tslice] - t0) * tscale, mixed_concs_ser[tslice, i], 
            lw=lw, color=clr, alpha=1.0, label=lbl)
ax.legend(frameon=False)
ax.set(xlabel="Time (s)", ylabel="Odor concentration")
ax.annotate(("Correlation:\n" + r"$\rho_{" + "{}{}".format(n_components-2, n_components-1) + "} =" 
            + "{:.1f}".format(correl_rho) + "$"), 
           xy=(0.0, mixed_concs_ser[tslice].max()*0.9))
fig.tight_layout(h_pad=0.0)
if do_save_plots:
    fig.savefig(pj(panels_folder, "mixed_turbulent_concentrations_sample.pdf"), transparent=True)
plt.show()
plt.close()

## Panel B: Performance vs correlation

In [ ]:
# Load saved statistics
all_jacs_stats = pd.read_hdf(os.path.join(data_folder, "correlation",
        "jaccard_similarities_stats_correlation_identity.h5"), key="df")
all_dists_stats = pd.read_hdf(os.path.join(data_folder, "correlation",
        "new_mix_distances_stats_correlation_identity.h5"), key="df")

In [ ]:
average_conc = np.sort(all_jacs_stats.index.get_level_values("new_conc").unique())[1]
n_new_concs = 2
keep_conc = np.sort(all_jacs_stats.index.get_level_values("new_conc").unique())[0:n_new_concs]
correl_range = np.sort(all_jacs_stats.index.get_level_values("correl_rho").unique())
fig, axes = plt.subplots(1, n_new_concs, sharex=True, sharey=True)
if n_new_concs == 1: axes = [axes]
else: axes = axes.flatten()
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*2.0, plt.rcParams["figure.figsize"][1])

# Order models according to the line order (best first)
show_models = ["optimal", "ibcm", "biopca", "avgsub", "none"]
model_zorder = ["none", "avgsub", "optimal",  "biopca", "ibcm"]
model_linestyles = {show_models[i]:neuron_styles[i % 6] for i in range(len(show_models))}
model_linestyles["ibcm"], model_linestyles["optimal"] = "-", model_linestyles["ibcm"]
for m in show_models[::-1]:  # Plot IBCM last
    if m == "none": continue  # clouds the graph too much
    for i in range(n_new_concs):
        new_conc = keep_conc[i]
        lower = (all_jacs_stats.loc[(m, correl_range, new_conc), "mean"] 
                 - np.sqrt(all_jacs_stats.loc[(m, correl_range, new_conc), "var"])).clip(lower=0.0)
        upper = (all_jacs_stats.loc[(m, correl_range, new_conc), "mean"] 
                 + np.sqrt(all_jacs_stats.loc[(m, correl_range, new_conc), "var"])).clip(upper=1.0)
        axes[i].fill_between(correl_range, lower, upper, color=model_colors.get(m), alpha=0.1)
for m in show_models:
    for i in range(n_new_concs):
        new_conc = keep_conc[i]
        axes[i].plot(correl_range, all_jacs_stats.loc[(m, correl_range, new_conc), "mean"], 
            label=model_nice_names.get(m, m), color=model_colors.get(m), alpha=1.0, 
            ls=model_linestyles[m], zorder=model_zorder.index(m) + 20, lw=1.25
        )
# Labeling the graphs, adding similarity between random odors, etc.
for i in range(n_new_concs):
    axes[i].set_title(r"New odor concentration $= {:.1f} \langle c \rangle$".format(keep_conc[i]/average_conc))
    axes[i].set_xlabel(r"Correlation $\rho_{23}$")
    axes[i].set_ylabel("Mean Jaccard similarity")
    ylim = axes[i].get_ylim()
    axes[i].set_ylim([ylim[0], 1.05])
leg_title = "Model"
axes[-1].legend(loc="upper left", bbox_to_anchor=(0.98, 1.), frameon=False, 
                title=leg_title, borderaxespad=0.0, handlelength=1.5)
fig.tight_layout()
if do_save_plots:
    fig.savefig(os.path.join(panels_folder, "jaccard_vs_correlation_performance.pdf"),
            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

## Panel C: example $h_\gamma$ series for IBCM model

In [ ]:
select_rhos = [-0.6, 0.2, 0.7, 1.0]
examples_hgammas_file = np.load(pj(data_folder, "correlation", "ibcm_examples_turbulent_correl.npz"))
examples_hgammas = {rho:examples_hgammas_file["cbars_gamma_ser_{}".format(rho)] for rho in select_rhos}

npoints, n_i_ibcm, n_b = examples_hgammas[select_rhos[0]].shape
duration = 360000.0
dt_u = 10.0  # ms
tser_example = np.arange(0.0, duration, duration / npoints)
t_axis = tser_example*dt_u/1000/60  # in minutes

In [ ]:
def compute_specifs(hgammaser):
    transient = 2*hgammaser.shape[0] // 3
    meandots = np.mean(hgammaser[transient:], axis=0)
    specifs = np.argmax(meandots, axis=1)
    return specifs

In [ ]:
# Compute specificity of each neuron in each simulation
gammas_specifs = {rho:compute_specifs(examples_hgammas[rho]) for rho in select_rhos}
for rho in select_rhos:
    print(rho)
    print(gammas_specifs[rho])

In [ ]:
# Neurons to highlight, changes for each plot to highlight different odor specificities
i_highlights_all = {
    -0.6: [0, 3, 1, 22],
    0.2: [4, 0, 9, 1], 
    0.4: [8, 1, 0, 7],
    0.7: [7, 0, 2],
    1.0: [2, 1, 18]
}

In [ ]:
# Version with one color per odor rather than per neuron
# Highlight three neurons
fig = plt.figure()
gs = fig.add_gridspec(4, 2*len(select_rhos)+1)
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*3, 
                    plt.rcParams["figure.figsize"][1])
ax0 = fig.add_subplot(gs[:, 0:2])
axes = [ax0] + [fig.add_subplot(gs[:, 2*i:2*i+2], sharey=ax0, sharex=ax0) 
                for i in range(1, len(select_rhos))]
axi = fig.add_subplot(gs[1:3, 2*len(select_rhos)])
ylims_shared = (-3.0, 15)

components_palette = list(sns.color_palette("colorblind", n_colors=n_b+1))[:-1] + ["xkcd:magenta"]
# One linestyle per neuron will help. 
linestyles_neurons = ["-", "--", "-.", ":"]

legend_styles = [[0,]*4, [0,]*4, [0,]*4, [0,]*4]

clr_back = back_palette[-1]
plot_skp = 25
tslice = slice(0, t_axis.shape[0], plot_skp)

# plot all other neurons first, skip some points
for irh, rho in enumerate(select_rhos):
    gammas_loc = gammas_specifs[rho]
    i_highlights = i_highlights_all[rho]
    ax = axes[irh]
    ax.set_ylim(ylims_shared)
    ax.set_title(r"$\rho_{23} = " + "{:.1f}".format(rho) + "$", y=0.96)
    ax.set_xlabel("Time (min)")
    if irh > 0:
        ax.yaxis.set_tick_params(which='both', labelleft=False)
    hgammaser_example = examples_hgammas[rho]
    for i in range(1, n_i_ibcm, 2):  # Plot half of them, no need for all
        if i in i_highlights: 
            continue
        else: 
            n_b_rho = n_b if abs(rho - 1.0) > 1e-6 else n_b-1
            for j in range(n_b_rho):
                ax.plot(t_axis[tslice], hgammaser_example[tslice, i, j], color=clr_back, 
                    ls="-", alpha=1.0, lw=plt.rcParams["lines.linewidth"])

    # Now plot the highlighted neuron
    n_b_rho = n_b if abs(rho - 1.0) > 1e-6 else n_b-1
    for j in range(n_b_rho):
        for i in range(len(i_highlights)):
            if abs(rho - 1.0) < 1e-6 and j == n_b_rho - 1 and i == 0:
                compcolor = components_palette[-1]
                complbl = "Odors {} + {}".format(n_b-2, n_b-1)
            else:
                compcolor = components_palette[j]
                complbl = None
            if j == gammas_loc[i_highlights[i]]:
                lw = plt.rcParams["lines.linewidth"]
                alph = 1.0
            else:
                lw = plt.rcParams["lines.linewidth"] - 0.3
                alph = 0.7
            li, = ax.plot(t_axis[tslice], hgammaser_example[tslice, i_highlights[i], j], 
                          color=compcolor, ls=linestyles_neurons[i], alpha=alph, 
                          lw=lw, label=complbl)
            if abs(rho - 1.0) > 1e-6:
                legend_styles[i][j] = li
    if abs(rho - 1.0) < 1e-6:
        ax.legend(frameon=False, loc="upper right", borderaxespad=0.1)

# Labels
axes[0].set_ylabel(r"Alignments $\bar{h}_{i\gamma} = \mathbf{\bar{m}}_i \cdot \mathbf{s}_{\gamma}$")

# Annotate with analytical results
#for j in range(2):
#    ax.axhline(analytical_hs_hn[j], lw=1.0, ls="-.", color="k")
#ax.annotate(r"Analytical $h_{\mathrm{ns}}$",  
#            xy=(t_axis[-5], analytical_hs_hn[1]-1.6),
#            ha="right", va="top", color="k", size=6)
#ax.annotate(r"",  
#            xy=(t_axis[-1]+1, analytical_hs_hn[1]-0.3), 
#            xytext=(t_axis[-5], analytical_hs_hn[1]-2.0),
#            ha="right", va="top", color="k", size=6, 
#            arrowprops={"color":"k", "headwidth":2.5, "headlength":2.0, "width":0.3})
#ax.annotate(r"Analytical $h_\mathrm{sp}$", xy=(t_axis[-5], analytical_hs_hn[0]+0.4), 
#            ha="right", va="bottom", color="k", size=6)

#ax.axhline(analytical_h_saddle, lw=1.0, ls=":", color="grey")
#ax.annotate(r"$h_\mathrm{saddle}$", xy=(t_axis[-150], analytical_h_saddle+0.1), 
#            ha="right", va="bottom", color="grey", size=6)

axi.tick_params(labelleft=True, labelbottom=False, labeltop=True)
i_highlights_dummy = np.arange(1, len(i_highlights_all[select_rhos[0]])+1)
for i in range(len(i_highlights_dummy)):
    for j in range(n_b):
        li = legend_styles[i][j]
        axi.plot([0.0+j, 0.5+j], [0.8*i, 0.8*i], color=li.get_color(), 
                 ls=li.get_linestyle(), alpha=1.0)
    axi.plot([0.0+n_b, 0.5+n_b], [0.8*i, 0.8*i], color=components_palette[-1], 
                 ls=li.get_linestyle(), alpha=1.0)
for side in ["bottom", "left", "top", "right"]:
    axi.spines[side].set_visible(False)
axi.tick_params(axis="both", length=0, pad=1)
axi.set_xticks(np.arange(0.25, n_b+1+0.25, 1.0))
#axi.set_xticklabels([r"$\mathbf{\bar{m}}^i \cdot \mathbf{s}_a$", r"$\mathbf{\bar{m}} \cdot \mathbf{s}_b$"])
axi.set_xticklabels(list(str(a) for a in range(0, n_b)) + ["2+3"])
axi.set_xlabel(r"Odor $\gamma$", size=6, labelpad=3)
axi.xaxis.set_label_position('top') 
axi.set_yticks([])
axi.invert_yaxis()
axi.set_yticklabels([])
#axi.set_yticklabels(["Neuron 2", "Neuron 1"])
axi.set_ylabel("Neuron", size=6, labelpad=0)

gs.tight_layout(fig, w_pad=0.0, h_pad=0.0)
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_correlation_ibcm_hgamma_series.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

## Panel D: BioPCA learning of this background (L, alignment, off-diagonal)
We may not have space to show the alignment error and the off-diagonal elements. 

In [ ]:
examples_pca_file = np.load(pj(data_folder, "correlation", "biopca_examples_turbulent_correl.npz"))
examples_pca_file.keys()

In [ ]:
examples_pca_file = np.load(pj(data_folder, "correlation", "biopca_examples_turbulent_correl.npz"))
examples_true_pca = {rho:examples_pca_file["true_pca_vals_{}".format(rho)] for rho in select_rhos}
examples_learnt_pca = {rho:examples_pca_file["learnt_pca_vals_{}".format(rho)] for rho in select_rhos}
examples_align_err = {rho:examples_pca_file["pca_align_error_{}".format(rho)] for rho in select_rhos}

n_comp = examples_learnt_pca[select_rhos[0]].shape[1]

In [ ]:
pca_palette = sns.color_palette("colorblind", n_colors=n_comp)
#pca_palette = sns.cubehelix_palette(start=0.1, rot=0.5, n_colors=n_comp)[::-1]
#pca_palette = np.asarray(sns.husl_palette(n_colors=n_comp, h=0.01, s=0.9, l=0.4, as_cmap=False))
#pca_palette = np.concat([pca_palette[::2], pca_palette[1::2]], axis=0)
sns.palplot(pca_palette)

In [ ]:
# First plot: eigenvalues
fig = plt.figure()
gs = fig.add_gridspec(4, 2*len(select_rhos)+1)
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*3, 
                    plt.rcParams["figure.figsize"][1])
ax0 = fig.add_subplot(gs[:, 0:2])
axes = [ax0] + [fig.add_subplot(gs[:, 2*i:2*i+2], sharey=ax0, sharex=ax0) 
                for i in range(1, len(select_rhos))]
axi = fig.add_subplot(gs[1:3, 2*len(select_rhos)])

skp_plot = 25
tslice = slice(0, tser_example.shape[0], skp_plot)
ylims_shared = [3e-4, 2.0]

# plot all other neurons first, skip some points
for irh, rho in enumerate(select_rhos):
    biopca_lines = []
    learnt_pca_values = examples_learnt_pca[rho]
    true_pca_values = examples_true_pca[rho]
    gammas_loc = gammas_specifs[rho]
    i_highlights = i_highlights_all[rho]
    ax = axes[irh]
    ax.set_ylim(ylims_shared)
    ax.set_title(r"$\rho_{23} = " + "{:.1f}".format(rho) + "$", y=0.96)
    ax.set(yscale="log", xlabel="Time (min)")
    if irh == 0:
        ax.set_ylabel("Principal values $(L_{ii}^{-1})$")
    if irh > 0:
        ax.yaxis.set_tick_params(which='both', labelleft=False)
    hgammaser_example = examples_hgammas[rho]
    for i in range(n_comp):
        li, = ax.plot(tser_example[tslice]*dt_u/1000/60, learnt_pca_values[tslice, i], label="Value {}".format(i),
                      lw=plt.rcParams["lines.linewidth"] - 0.5*i/n_comp, zorder=10-i, color=pca_palette[i])
        biopca_lines.append(li)
        if true_pca_values[i] / true_pca_values.max() > 1e-12:
            ax.axhline(true_pca_values[i], ls="--", color=pca_palette[i], 
                       lw=plt.rcParams["lines.linewidth"] - 0.5*i/n_comp, zorder=n_comp-i)

# Custom legend to indicate analytical vs learnt?
ax = axes[0]
handles = [mpl.lines.Line2D([0], [0], color="grey", ls="-", label=r"BioPCA", 
                            lw=plt.rcParams["lines.linewidth"]), 
          mpl.lines.Line2D([0], [0], color="grey", ls="--", label="True PCA", 
                          lw=plt.rcParams["lines.linewidth"])]
leg = ax.legend(handles=handles, frameon=False, ncols=2, columnspacing=0.8, borderaxespad=0)
leg.set_zorder(30)

# Small axis for second legend for LN indices
axi.tick_params(labelleft=True, labelbottom=False, labeltop=True)
for j in range(n_comp):
    li = biopca_lines[j]
    axi.plot([0.0, 1.0], [0.8*j, 0.8*j], color=li.get_color(), label=str(j),
                 ls=li.get_linestyle(), alpha=li.get_alpha(), lw=li.get_linewidth())
handles, labels = axi.get_legend_handles_labels()
axi.clear()
leg = axi.legend(handles, labels, title="LN index", title_fontsize=6, 
                 frameon=False, loc="upper left", borderaxespad=0.0)
axi.axis("off")


fig.tight_layout(w_pad=0.0, h_pad=0.0)
if do_save_plots:
    fig.savefig(os.path.join(panels_folder, "supfig_correlation_biopca_eigenvalues.pdf"), 
            transparent=True, bbox_inches="tight", bbox_extra_artists=(leg,))
plt.show()
plt.close()